# Implementación del PRM (Probabilistic Roadmap)

Propuesto por Kavraki y otros [1], el PRM (del inglés Probabilistic Roadmap) es un algoritmo de planificación basado en muestreo. Éste basa su funcionamiento en construir un grafo de caminos en el espacio libre y despúes utilizar este mismo grafo para encontrar las rutas.

En este programa se implementan las dos etapas del PRM, la construcción y la consulta, para la cual se usa la heurística A*.

1. Load the obstacle map data
2. Sample nodes (use KDTrees here)
3. Connect nodes (use KDTrees here)
4. Visualize graph
5. Define heuristic
6. Define search method
7. Execute and visualize

We'll load the data for you and provide a template for visualization.

[1] Kavraki, L. E., Svestka, P., Latombe, J. C., & Overmars, M. H. (1996). Probabilistic roadmaps for path planning in high-dimensional configuration spaces. IEEE transactions on Robotics and Automation, 12(4), 566-580.

In [ ]:
# Verificar que tengamos networkx
# import sys
#!{sys.executable} -m pip install -I networkx==2.1
import pkg_resources
#pkg_resources.require("networkx==2.1")
import networkx as nx

#!pip install sklearn

In [ ]:
nx.__version__ # should be 2.1

In [ ]:
# Importar paquetes
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, LineString
from queue import PriorityQueue
from sklearn.neighbors import KDTree
import time
# funciones previamente programadas
import muestreo 

%matplotlib inline
plt.rcParams['figure.figsize'] = 14, 14

In [ ]:
# Cargamos los datos utilizados en el programa anterior.
filename = 'data_reduced.csv'
data = np.loadtxt(filename, delimiter=',', dtype='float64', skiprows=2)

## Construcción
### Muestreo aleatorio


In [ ]:
def collides(polygons, tree, point):   
    # Tarea: Determina si un punto está en colision con algún polígono
    collide = False
    x,y,z = point
    # guarda en p un Punto definido por shapely
    p = None
    
    # Tarea: Encuentra el centro del polígono más cercano
    p_array = np.array([x,y])
    idx = tree.query([p_array], k=1, return_distance = False)[0]
    
    poly = polygons[int(idx)]
    polygon, height = poly
    if polygon.contains(p):
        if z < height:
            collide = True
            
    return collide

In [ ]:
# Se utilizan las funciones de muestreo ya programadas
samples = muestreo.get_samples(data)
print(samples[:5])

# Create the centers of the polygons
polygons, centers = muestreo.extract_polygons_and_centers(data)
print(centers[:10])



In [ ]:
# then use KDTree to find nearest neighbor polygon and test for collision
# Tip: use un KDTree
tree = KDTree(centers)

In [ ]:
# Mantener solo las muestras que no están en colisión
t0 = time.time()
to_keep = []
for point in samples:
    # Verificamos colisión
    if not collides(polygons, tree, point):
        to_keep.append(point)
time_taken = time.time() - t0
print("Time taken {0} seconds ...", time_taken)

In [ ]:
print(len(to_keep), "muestras libres de colisión")

### Conexión de nodos

Now we have to connect the nodes. There are many ways they might be done, it's completely up to you. The only restriction being no edge connecting two nodes may pass through an obstacle.

NOTE: You can use `LineString()` from the `shapely` library to create a line. Additionally, `shapely` geometry objects have a method `.crosses` which return `True` if the geometries cross paths, for instance your `LineString()` with an obstacle `Polygon()`!

**Suggested method**

1. Cast nodes into a graph called "g" using networkx
2. Write a method "can_connect()" that:
    1. casts two points as a shapely LineString() object
    2. tests for collision with a shapely Polygon() object
    3. returns True if connection is possible, False otherwise
3. write a method "create_graph()" that:
    1. defines a networkx graph as g = Graph()
    2. defines a tree = KDTree(nodes)
    3. test for connectivity between each node and 
    4. of it's nearest neighbors
    5. if nodes are connectable, add an edge to graph
    6. Iterate through all candidate nodes!

In [ ]:
def can_connect(p1, p2, obstacles):
    x1,y1,z1 = p1
    x2,y2,z2 = p2
    
    # Construye una linea 2D
    # https://shapely.readthedocs.io/en/stable/manual.html#linestrings
    line = None
    free = True
    
    # Para cada uno de los obstáculos 
        # descomponer el obstáculo en su polígono y altura
        # Verificar si la linea cruza el polígono
            #Establecer a false
    
    return free

In [ ]:
def create_graph(milestones, polygons, k):
    array_milestones = np.array(milestones)
    #print(array_milestones[:10])
    m_tree = KDTree(array_milestones)
    g = nx.Graph()
    
    for mil in milestones:
        #print(mil)
        mil_array = np.array(mil)
        #print(mil_array)
        
        # find the nearest k milestones
        distances, indexes = m_tree.query([mil_array], k)

        #print(dist)
        #print(idx)
        for dist, idx in zip(distances[0], indexes[0]):
            #print(dist)
            #print(idx)
            if dist > 0:
                if can_connect(mil, milestones[int(idx)], polygons):
                    # Agregar una arista a partir de "mil, milestones[int(idx)], weight=dist"
                    #https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.add_edge.html
        
    return g

In [ ]:
# TODO: create the graph
# Define un k
k = None
g = create_graph(to_keep, polygons, k)

### Visualización

In [ ]:
# Create a grid map of the world
from grid import create_grid
# This will create a grid map at 1 m above ground level
grid = create_grid(data, 1, 1)

fig = plt.figure()

plt.imshow(grid, cmap='Greys', origin='lower')

nmin = np.min(data[:, 0])
emin = np.min(data[:, 1])

# If you have a graph called "g" these plots should work
# Draw edges
for (n1, n2) in g.edges:
    plt.plot([n1[1] - emin, n2[1] - emin], [n1[0] - nmin, n2[0] - nmin], 'black' , alpha=0.5)

# Draw all nodes connected or not in blue
for n1 in to_keep:
    plt.scatter(n1[1] - emin, n1[0] - nmin, c='red')
    
# Draw connected nodes in red
for n1 in g.nodes:
    plt.scatter(n1[1] - emin, n1[0] - nmin, c='green')
    


plt.xlabel('NORTH')
plt.ylabel('EAST')

plt.show()

## Consulta de ruta

### Implementación de A*

Deinición de la heuristica

In [ ]:
import numpy.linalg as LA

def heuristic(n1, n2):
    # Tarea: Definir la distancia a usar
    # Ti: https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html 
    return None

Algoritmo de búsqueda

In [ ]:
def a_star(graph, heuristic, start, goal):
    """Modified A* to work with NetworkX graphs."""

    path = []
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)

    branch = {}
    found = False
    
    while not queue.empty():
        item = queue.get()
        current_cost = item[0]
        current_node = item[1]

        if current_node == goal:        
            print('Found a path.')
            found = True
            break
        else:
            for next_node in graph[current_node]:
                cost = graph.edges[current_node, next_node]['weight']
                new_cost = current_cost + cost + heuristic(next_node, goal)
                
                if next_node not in visited:                
                    visited.add(next_node)               
                    queue.put((new_cost, next_node))
                    
                    branch[next_node] = (new_cost, current_node)
             
    path = []
    path_cost = 0
    if found:
        
        # retrace steps
        path = []
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][1])
            n = branch[n][1]
        path.append(branch[n][1])
            
    return path[::-1], path_cost


In [ ]:
start = list(g.nodes)[0]
k = np.random.randint(len(g.nodes))
print(k, len(g.nodes))
goal = list(g.nodes)[k]
print('start', start)
print('goal', goal)

path, cost = a_star(g, heuristic, start, goal)
print(len(path), path)

path_pairs = zip(path[:-1], path[1:])
for (n1, n2) in path_pairs:
    print(n1, n2)

# Prueba y visualización

In [ ]:
fig = plt.figure()

plt.imshow(grid, cmap='Greys', origin='lower')

nmin = np.min(data[:, 0])
emin = np.min(data[:, 1])
   
# draw nodes
for n1 in g.nodes:
    plt.scatter(n1[1] - emin, n1[0] - nmin, c='green')
    
# draw edges
for (n1, n2) in g.edges:
    plt.plot([n1[1] - emin, n2[1] - emin], [n1[0] - nmin, n2[0] - nmin], 'grey')
    
# TODO: add code to visualize the path
path_pairs = zip(path[:-1], path[1:])
for (n1, n2) in path_pairs:
    plt.plot([n1[1] - emin, n2[1] - emin], [n1[0] - nmin, n2[0] - nmin], 'orange', linewidth=3)
    


plt.xlabel('NORTH')
plt.ylabel('EAST')

plt.show()